In [62]:
import re
from collections import defaultdict as dd
import random
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from src.models.word2vec import Word2Vec

from src.models.papugapt import PapuGaPT2
from src.models.questions_clusterer.model import QuestionsClusterer
from src.models.questions_clusterer.w2v_clusterer import read_qa_tsv, Word2VecClusterer
from scipy import spatial

In [43]:
with open("../../../data/QA/2021-question-answering/dev-0/expected.tsv", 'r') as f:
    answers = f.readlines()
    answers = [a.strip().split("\t") for a in answers]

with open("../../../data/QA/2021-question-answering/dev-0/in.tsv", 'r') as f:
    questions = f.readlines()

QA_data = [{'question': questions[i], 'answers': answers[i]} for i in range(len(answers))]

In [44]:
len(QA_data)

1000

In [45]:
with open("../../../data/QA/2021-question-answering/test-A/expected.tsv", 'r') as f:
    answers_test = f.readlines()
    answers_test = [a.strip().split("\t") for a in answers_test]

with open("../../../data/QA/2021-question-answering/test-A/in.tsv", 'r') as f:
    questions_test = f.readlines()

QA_data_test = [{'question': questions_test[i], 'answers': answers_test[i]} for i in range(len(answers_test))]

In [46]:
len(QA_data_test)

2500

In [96]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
import os

# QUESTIONS_ANSWERS_PATH = Path(__file__).parents[2] / 'data' / 'questions_answers' / 'def_question.tsv'
QUESTIONS_ANSWERS_PATH = Path(os.getcwd()) / 'data' / 'QA' / '2021-question-answering' / 'dev-0' / 'in.tsv'
GENERATION_CONFIG = {
    "do_sample": True,
    "top_k": 100,
    "top_p": 0.75,
    "max_new_tokens": 5,
    "num_beams": 2,
    "no_repeat_ngram_size": 5,
}
NUM_CLUSTERS = 15
QA_SAMPLED_FROM_CLUSTER = 10
questions_answerer = PapuGaPT2()

In [56]:
w2v = Word2Vec()

In [58]:
w2v.get_embedding('Islandia jest większa or Irlandii')

array([-1.8209262 ,  0.46863934, -0.02018233,  0.14734931,  1.4696263 ,
        0.8995757 ,  0.65541464,  0.7323353 , -0.07630966, -0.30864224,
       -0.620011  ,  0.31744894,  1.8920666 ,  0.687854  ,  2.17895   ,
        0.17316031, -0.40522766,  0.22720933, -0.09203736,  1.5310496 ,
        0.980578  ,  0.893286  ,  1.091339  , -0.36462498,  1.0545944 ,
        2.1546147 ,  1.0009694 , -4.1941323 ,  2.2078128 ,  1.8071927 ,
        1.8270999 , -1.2376189 , -2.0329592 , -0.77817273, -0.4379066 ,
       -0.9732259 ,  0.02672664,  0.10094202, -0.9594877 , -0.83171225,
        0.840192  , -0.18930133,  0.57975703, -0.959116  ,  1.8593124 ,
        0.5756753 , -2.8768356 , -1.7704586 , -2.0477638 , -0.79411894,
        0.13434134,  2.5632596 ,  3.7663238 , -1.8276887 ,  2.1867616 ,
       -0.02511132,  1.7058792 , -0.69824964, -0.9910396 , -3.2143862 ,
       -1.6143607 ,  0.12167533,  1.391776  , -1.2362894 , -1.3193991 ,
        1.2411398 ,  1.2661034 , -0.89193994,  0.30009964,  1.76

In [61]:
# heurystyka 1: pytania tak / nie

yes_no_train = list()
for d in QA_data:
    if re.match("Czy ", d['question']) and not re.match(r"Czy .* czy", d['question']):
        if d['answers'][0] not in ['tak', 'nie']:
             print(d['question'])
        else:
            qq = d['question'].replace('Czy ',  '').replace('?\n',  '')
            w2v.get_embedding(qq)
            yes_no_train.append((f"{qq}:  {d['answers'][0]}\n", w2v.get_embedding(qq)))

In [73]:
emb = w2v.get_embedding(qq)

cosines = [1 - spatial.distance.cosine(emb, d[1]) for d in yes_no_train]
pd.Series(cosines).abs().sort_values().iloc[:5].index.values

/home/kabalce/anaconda3/envs/qa/lib/python3.10/site-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


array([41, 25, 34, 30, 77])

In [85]:
noq = 0
well_done = 0
for  d  in QA_data_test:
    if re.match("Czy ", d['question']) and not re.match(r"Czy .* czy", d['question']):
        qq = d['question'].replace('Czy ',  '').replace('?\n',  '')
        emb = w2v.get_embedding(qq)

        cosines = [1 - spatial.distance.cosine(emb, d[1]) for d in yes_no_train]
        similar_questions = pd.Series(cosines).abs().sort_values().iloc[:8].index.values

        yn_prompt = "".join([yes_no_train[i][0] for i in similar_questions])

        qq = d['question'].replace('Czy ',  '').replace('?\n',  '')
        prompt="".join([yn_prompt, f"{qq}: "])
        # print(prompt)
        response = questions_answerer.respond_to_prompt(
            prompt=prompt,
            generation_config=GENERATION_CONFIG,
            end_sequence="###",
        )

        # TODO: sprawdź 'nie', w  niejednoznacznych odpowiadaj: tak
        ans = True
        if 'nie'  in response[0].replace(prompt, '').lower():
            ans  = False

        print(f"Model: {ans}; True answer: {d['answers'][0]  ==  'tak'}")
        well_done += ans == (d['answers'][0]  ==  'tak')
        noq += 1

print(well_done / noq)

/home/kabalce/anaconda3/envs/qa/lib/python3.10/site-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Model: False; True answer: True
Model: False; True answer: True
Model: False; True answer: True
Model: False; True answer: True
Model: False; True answer: False
Model: True; True answer: True
Model: False; True answer: True
Model: False; True answer: False
Model: False; True answer: False
Model: True; True answer: True
Model: False; True answer: True
Model: False; True answer: True
Model: False; True answer: True
Model: True; True answer: True
Model: False; True answer: True
Model: True; True answer: True
Model: False; True answer: False
Model: False; True answer: True
Model: False; True answer: False
Model: False; True answer: False
Model: False; True answer: True
Model: False; True answer: True
Model: False; True answer: False
Model: False; True answer: False
Model: False; True answer: True
Model: True; True answer: True
Model: False; True answer: True
Model: False; True answer: True
Model: False; True answer: True
Model: True; True answer: True
Model: True; True answer: True
Model: 

In [86]:
# heurystyka 2: jaki / który

which_train = dd(list)
for d in QA_data:
    if re.match("któr", d['question'].split(" ")[1]) or re.match("jak", d['question'].split(" ")[1]):
        which_train[d['question'].split(" ")[2]].append((d['question'], d['answers']))
    elif  re.match("któr", " ".join(d['question'].split(" ")[:2]).lower()):
        which_train[d['question'].split(" ")[1]].append((d['question'], d['answers']))
    elif  re.match("któr", " ".join(d['question'].split(" ")[:3]).lower()):
        which_train[d['question'].split(" ")[2]].append((d['question'], d['answers']))
    elif  re.match("któr", " ".join(d['question'].split(" ")[:4]).lower()):
        which_train[d['question'].split(" ")[3]].append((d['question'], d['answers']))
    elif re.match("jak.* nazyw.* się", " ".join(d['question'].split(" ")[:2]).lower()):
        which_train[d['question'].split(" ")[3]].append((d['question'], d['answers']))
    elif re.match(r"jak[a-z]* (?!nazyw)", " ".join(d['question'].split(" ")[:2]).lower()):
        which_train[d['question'].split(" ")[1]].append((d['question'], d['answers']))
which_train

defaultdict(list,
            {'państwie': [('W którym państwie rozpoczyna się akcja powieści „W pustyni i w puszczy”?\n',
               ['w Egipcie']),
              ('W którym państwie leży Bombaj?\n', ['w Indiach']),
              ('W którym państwie leży Klondike, obszar znany z wydobycia złota?\n',
               ['w Kanadzie']),
              ('W którym państwie najwyższym odznaczeniem jest krzyż Wiktorii?\n',
               ['w Wielkiej Brytanii']),
              ('W którym państwie w latach 50-tych XX wieku wydana została pierwsza karta kredytowa?\n',
               ['w Stanach Zjednoczonych', 'w USA']),
              ('W którym państwie w 1958 r. rozpoczęła się kampania polityczna nazwana Wielkim Skokiem?\n',
               ['w Chinach']),
              ('W którym państwie leży Brno?\n', ['w Czechach']),
              ('W którym państwie powstał taoizm?\n', ['w Chinach']),
              ('W którym państwie ma główną siedzibę agencja prasowa DPA?\n',
               ['w Niemcze

In [111]:
GENERATION_CONFIG['max_new_tokens'] = 5

# Other idea: use only one example question, with the answer most similar to current questiuon

for d in QA_data_test:
    if re.match("któr", d['question'].split(" ")[1]) or re.match("jak", d['question'].split(" ")[1]):
        i = 2
    elif  re.match("któr", " ".join(d['question'].split(" ")[:2]).lower()):
        i = 1
    elif  re.match("któr", " ".join(d['question'].split(" ")[:3]).lower()):
        i = 2
    elif  re.match("któr", " ".join(d['question'].split(" ")[:4]).lower()):
        i = 3
    elif re.match("jak.* nazyw.* się", " ".join(d['question'].split(" ")[:2]).lower()):
        i = 3
    elif re.match(r"jak[a-z]* (?!nazyw)", " ".join(d['question'].split(" ")[:2]).lower()):
        i = 1
    else:
        continue

    prompt = "\n".join([" ".join(reversed(q[0].replace(r"Któr\S* ",  "").replace(r"Jak\S* ",  "").replace(r".* któr\S* ",  "").replace(r".* jak\S* ",  "").replace("\n",  "").split(" "))) + ":  " + q[1][0] for q in which_train[d['question'].split(" ")[i]]]) + "\n" +  " ".join(reversed(d['question'].replace(r"Któr\S* ",  "").replace(r"Jak\S* ",  "").replace(r".* któr\S* ",  "").replace(r".* jak\S* ",  "").replace("\n",  "").split(" "))) + ":  "

    response = questions_answerer.respond_to_prompt(
            prompt=prompt,
            generation_config=GENERATION_CONFIG,
            end_sequence="###",
    )

    prompt2 = "\n".join([q[0].replace(r"Któr\S* ",  "").replace(r"Jak\S* ",  "").replace(r".* któr\S* ",  "").replace(r".* jak\S* ",  "").replace("\n",  "") + ":  " + q[1][0] for q in which_train[d['question'].split(" ")[i]]]) + "\n" +  d['question'].replace(r"Któr\S* ",  "").replace(r"Jak\S* ",  "").replace(r".* któr\S* ",  "").replace(r".* jak\S* ",  "").replace("\n",  "") + ":  "

    response2 = questions_answerer.respond_to_prompt(
            prompt=prompt2,
            generation_config=GENERATION_CONFIG,
            end_sequence="###",
        )

    print(d['question'].replace("\n", ""))
    print(d['answers'])
    print(response[0].replace(prompt, "").replace("\n", " "))
    print(response2[0].replace(prompt2, "").replace("\n", " "))
    print("\n\n")

W którym państwie została ogłoszona „Deklaracja praw człowieka i obywatela”?
['we Francji']
 w Indiach W.
 w USA W którym



Która kawa zawiera alkohol: po turecku czy po irlandzku?
['po irlandzku']
 "Posiadłość:
  4        



W którym mieście zmarł Sławomir Mrożek?
['w Nicei']
 w Krakowie,  w
 w Warszawie  W



W którym polskim mieście wybrzeża znajduje się Fontanna Neptuna?
['w Gdańsku']
 w „Janosikowym
 w „Kabare



Na którym kontynencie leżała Kartagina?
['w Afryce']
 w Afryce Południowej  
 w Afryce   



Które owady australijskie tworzą ogromne kopce, wysokie na 6 metrów?
['termity']
 Nie ma chyba bardziej
 W tym roku w



W którym zespole ludowym w początkach kariery śpiewała Irena Santor?
['Mazowsze']
 Mazowsze W:
 Mazowsze W jakim



W którym mieście znajduje się cmentarz na Rossie?
['w Wilnie']
 w Krakowie Ponia
 w Warszawie  W



Jak miał na imię generał amerykański McArthur?
['Douglas']
 Lucien Mar
 William B. Brown 



Zwolnienie której robotnicy spowodowało strajk w Stoc

KeyboardInterrupt: 

In [83]:
# what is called questions
# idea: look at the embeddings of keywords. Construct prompt using similar keywords

names = dd(list)
for d in QA_data:
    if re.match("Jak nazyw", d['question']):
        names[" ".join(d['question'].split(" ")[3:5])].append(d['answers'])
names

# could look for synonims (similar embeddings)

defaultdict(list,
            {'pierwsza litera': [['alfa']],
             'dowolny odcinek': [['cięciwa']],
             'pierwiastek o': [['becquerel']],
             'sztywny kapelusz': [['melonik']],
             'wypukła albo': [['menisk']],
             'fizyk, który': [['Aleksander Graham Bell']],
             'wzgórze w': [['Montmartre']],
             'trucizna przyrządzona': [['cykuta']],
             'stolica Kuby?\n': [['Hawana']],
             'agent brytyjskiego': [['James Bond']],
             'przyrząd do': [['fajka']],
             'potomek osadników': [['Kreol']],
             'mistrzyni Polski': [['Helena Marusarzówna']],
             'grecki bóg,': [['Hermes']],
             'tradycyjna żydowska': [['jarmułka',
               'kipa',
               'sztrajmł',
               'sztrajmel',
               'sztrejmł']],
             'najwyższy szczyt': [['Radziejowa']],
             'przodownik chóru': [['koryfeusz']],
             'jednostka w': [['dywizjon', 'pułk lot

In [89]:
for d in QA_data:
    if re.match(r".* czy ", d['question']):
        if sum([d['answers'][0] in p for p in  d['question'].split(" czy ")]) == 0:
            print(d['question'].split("czy"), d['answers'])

['Co ile lat odradzał się z popiołów mityczny feniks – co 50, co 500 ', ' co 5000 lat?\n'] ['co 500 lat']
['Miesiąc miodowy to okres przed ', ' po ślubie?\n'] ['przed ślubem']


In [123]:
for d in QA_data:
    if re.match("Ile ", d['question']):
        for a in d['answers']:
            cc = 0
            try:
                float(a.replace(",", "."))
            except:
                cc += 1
        if cc == len(d['answers']):
            print(d['answers'])

['beczkę']
['6 niedziel']
['21 dni']


In [126]:
for d in QA_data:
    if re.match("Kto ", d['question']) or re.match(".* kto ", d['question']) or re.match("Kogo ", d['question']) or re.match(".* kto ", d['question']):
        print(d['answers'])

['Paul Gauguin']
['Roman Kłosowski']
['Katarzyna Cichopek']
['prezydent']
['Julian Tuwim']
['David Livingstone']
['Hades']
['Marek Kondrat']
['Jan III Sobieski']
['Jean Foucault']
['Marcin Luter']
['Sejm']
['Stefan Żeromski']
['Walt Disney']
['Jankiel']
['admirał floty']
['Jadwiga']
['Jan bez Ziemi']
['Władysław Reymont']
['Rudolf Diesel']
['Piotr Czajkowski']
['Ignacy Krasicki']
['Mikołaj Rej']
['Jan Sobieski']
['Roald Amundsen']
['od miecza']
['Kościuszko']
['Matka Boska']
['Louis Braille']
['Karol IV Luksemburski']
['Jan Brzechwa']
['Józefa Piłsudskiego']
['Julian Tuwim']
['Jan Brzechwa']
['Jerzy Andrzejewski']
['Julian Tuwim']
['Tennessee Williams']
['John Glenn']
['Jan Zamoyski']
['Włodzimierz Wolski']
['Walt Disney']
['Orfeusz']
['Anteusz']
['Marlon Brando']
['Elżbieta II', 'królowa brytyjska']
['Stanisław Mikołajczyk']
['Hypnos']
['Posejdon']
['Mikołaj Kopernik']
['Isaac Newton']
['Juliusz Słowacki']
['wikingowie']
['Ingmar Bergman']
['lis']
['Adam Mickiewicz']
['Franciszek Karp

In [20]:
counter = 0
q_no_cat = []
for d in QA_data:
    if re.match("Czy ", d['question']) and not re.match(r"Czy .* czy", d['question']):
        continue
    if re.match(r".* czy ", d['question']):
        continue
    if re.match("Ile ", d['question']):
        continue
    if re.match("któr", d['question'].split(" ")[1]) or re.match("jak", d['question'].split(" ")[1]):
        continue
    if  re.match("któr", " ".join(d['question'].split(" ")[:4]).lower()):
        continue
    if re.match("jak nazywa się", " ".join(d['question'].split(" ")[:2]).lower()):
        continue
    if re.match("Jak nazyw", d['question']):
        continue
    if re.match(r"jak[a-z]* (?!nazyw)", " ".join(d['question'].split(" ")[:2]).lower()):
        continue
    if re.match("Kto ", d['question']) or re.match(".* kto ", d['question']) or re.match("Kogo ", d['question']) or re.match(".* kto ", d['question']):
        continue
    print(f"{counter}. {d['question']}")
    q_no_cat.append(d['question'])
    counter += 1

0. Co budował w Egipcie inżynier Tarkowski, ojciec Stasia?

1. Kwartet – to ilu wykonawców?

2. Co znaczy pochodzące z francuskiego słowo aliant?

3. Co znaczą słowa „Quo vadis”?

4. Proszę rozwinąć skrót powstałego w 1918 roku urzędu GUS.

5. Dokończ przysłowie: „dobry zwyczaj: nie...”?

6. Gdzie miały miejsce objawienia z 1917 roku?

7. W zdaniu: „Szybko przeskoczył przez płot” który wyraz jest przysłówkiem?

8. Tzw. suchy lód to stała postać którego związku chemicznego?

9. Co z greckiego oznacza ochlokracja?

10. Gdzie w październiku 1813 roku stoczono bitwę przegraną przez Napoleona?

11. Co produkowała wytwórnia astronoma Jana Heweliusza?

12. Co wykluwa się z żabiego skrzeku?

13. Mistrzostwa świata w której dyscyplinie bronił w 2013 roku Sergio Martinez?

14. Kardynał Richelieu był sekretarzem którego francuskiego króla?

15. „Coś Ty Atenom zrobił, Sokratesie” – czyjego autorstwa jest to utwór?

16. Z mleka którego zwierzęcia wyrabiana jest serwatka zwana żętycą?

17. Do kogo s

In [21]:
q_no_cat

['Co budował w Egipcie inżynier Tarkowski, ojciec Stasia?\n',
 'Kwartet – to ilu wykonawców?\n',
 'Co znaczy pochodzące z francuskiego słowo aliant?\n',
 'Co znaczą słowa „Quo vadis”?\n',
 'Proszę rozwinąć skrót powstałego w 1918 roku urzędu GUS.\n',
 'Dokończ przysłowie: „dobry zwyczaj: nie...”?\n',
 'Gdzie miały miejsce objawienia z 1917 roku?\n',
 'W zdaniu: „Szybko przeskoczył przez płot” który wyraz jest przysłówkiem?\n',
 'Tzw. suchy lód to stała postać którego związku chemicznego?\n',
 'Co z greckiego oznacza ochlokracja?\n',
 'Gdzie w październiku 1813 roku stoczono bitwę przegraną przez Napoleona?\n',
 'Co produkowała wytwórnia astronoma Jana Heweliusza?\n',
 'Co wykluwa się z żabiego skrzeku?\n',
 'Mistrzostwa świata w której dyscyplinie bronił w 2013 roku Sergio Martinez?\n',
 'Kardynał Richelieu był sekretarzem którego francuskiego króla?\n',
 '„Coś Ty Atenom zrobił, Sokratesie” – czyjego autorstwa jest to utwór?\n',
 'Z mleka którego zwierzęcia wyrabiana jest serwatka zwan

In [2]:
# import pickle
# with open('QA_data.pkl', 'wb') as f:
#     pickle.dump(QA_data, f)

NameError: name 'QA_data' is not defined

In [3]:
import random
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from src.models.papugapt import PapuGaPT2
from src.models.questions_clusterer.model import QuestionsClusterer
from src.models.questions_clusterer.w2v_clusterer import read_qa_tsv, Word2VecClusterer

In [16]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
import os
# QUESTIONS_ANSWERS_PATH = Path(__file__).parents[2] / 'data' / 'questions_answers' / 'def_question.tsv'
QUESTIONS_ANSWERS_PATH = Path(os.getcwd()) / 'data' / 'QA' / '2021-question-answering' / 'dev-0' / 'in.tsv'
GENERATION_CONFIG = {
    "do_sample": True,
    "top_k": 100,
    "top_p": 0.75,
    "max_new_tokens": 15,
    "num_beams": 2,
    "no_repeat_ngram_size": 5,
}
NUM_CLUSTERS = 15
QA_SAMPLED_FROM_CLUSTER = 10

In [3]:
questions_answerer = PapuGaPT2()

In [4]:
response = questions_answerer.respond_to_prompt(
        prompt='Królowa Jadwiga zmarła w wieku',
        generation_config=GENERATION_CONFIG,
        end_sequence="###",
    )
response

['Królowa Jadwiga zmarła w wieku 99 lat. Jej pogrzeb odbył się na Wawelu']

In [7]:
import pickle
import re
with open('QA_data.pkl', 'rb') as f:
    QA_data = pickle.load(f)

EOFError: Ran out of input

In [8]:
for d in QA_data:
    if re.match("Czy ", d['question']) and not re.match(r"Czy .* czy", d['question']):
        print(d['question'])
        if d['answers'][0] not in ['tak', 'nie']:
            continue
        else:
            ans = questions_answerer.respond_to_prompt(
                prompt=d['question'].replace("Czy ", "...").replace("?\n", ""),
                generation_config=GENERATION_CONFIG,
                end_sequence="###",
            )
            print(ans)

NameError: name 'QA_data' is not defined

In [ ]:
# " ".join(d["question"].split(" ")[3: ]).replace("?\n", "")

In [ ]:
# TODO: napisz lepszy prompt

for d in QA_data:
    if re.match("któr", d['question'].split(" ")[1]) or re.match("jak", d['question'].split(" ")[1]):
        print(d['question'].replace("\n", " "))
        ans = questions_answerer.respond_to_prompt(
                prompt=" ".join(reversed(d["question"].split(" ")[2: ])).replace("?\n", ""),
                generation_config=GENERATION_CONFIG,
                end_sequence="###",
            )
        print(" ".join(ans[0].split(" ")[len(d["question"].split(" ")[2: ]):]))
        print(d['answers'])
        print("\n")

W którym państwie rozpoczyna się akcja powieści „W pustyni i w puszczy”? 
w którym nie ma ludzi, a ludzie są na pustyni”.
W
['w Egipcie']


W jakim zespole występowała Hanka w filmie „Żona dla Australijczyka”? 
który powstał w 1995 roku. „Żona filmie” to film,
['Mazowsze']


W którym państwie leży Bombaj? 
nad Morzem Czerwonym, w środkowej części półwyspu Synaj, nad
['w Indiach']


Hymnem którego kraju jest pieśń „Gwiaździsty sztandar”? 
w którym nie ma miejsca dla ludzi, którzy nie mają prawa głosu”.
['Stanów Zjednoczonych', 'USA']


W jakim stylu wykonany został nagrobek Kazimierza Jagiellończyka? 
barokowym. Napis na nagrobku głosi: „Zamieszkali w
['gotyckim']


W którym wieku powstał na Widawie most św. Karola? 
XIV w. z fundacji króla Zygmunta III Wazy. Przebudowany w stylu
['w XIV', '14']


W którym mieście zginęła w wypadku samochodowym księżna Diana? 
To był wypadek, w którym zginęła księżna Diana.
-
['w Paryżu']


Na którym kontynencie występuje ryś iberyjski? 
afrykańskim. W 

In [ ]:
for d in QA_data:
    if re.match("któr", d['question'].split(" ")[1]) or re.match("jak", d['question'].split(" ")[1]):
        print(d['question'].replace("\n", " "))
        ans = questions_answerer.respond_to_prompt(
                prompt=" ".join(reversed(d["question"].split(" ")[2: ])).replace("?\n", ""),
                generation_config=GENERATION_CONFIG,
                end_sequence="###",
            )
        print(ans)
        print(d['answers'])
        print("\n")

['Giżycko leży województwie warmińsko-mazurskim, w powiecie giżyckim']


In [ ]:
# heurystyka 1: pytania tak / nie
for d in QA_data:
    if re.match("Czy ", d['question']) and not re.match(r"Czy .* czy", d['question']):
        if d['answers'][0] not in ['tak', 'nie']:
            print(d['question'])

types = dd(list)
for d in QA_data:
    if re.match("któr", d['question'].split(" ")[1]) or re.match("jak", d['question'].split(" ")[1]):
        types[d['question'].split(" ")[2]].append(d['answers'])
    elif re.match("któr", " ".join(d['question'].split(" ")[:2]).lower()):
        types[d['question'].split(" ")[1]].append(d['answers'])
    elif re.match("któr", " ".join(d['question'].split(" ")[:3]).lower()):
        types[d['question'].split(" ")[2]].append(d['answers'])
    elif re.match("któr", " ".join(d['question'].split(" ")[:4]).lower()):
        types[d['question'].split(" ")[3]].append(d['answers'])
    elif re.match("jak nazywa się", " ".join(d['question'].split(" ")[:2]).lower()):
        types[d['question'].split(" ")[3]].append(d['answers'])
    elif re.match(r"jak[a-z]* (?!nazyw)", " ".join(d['question'].split(" ")[:2]).lower()):
        types[d['question'].split(" ")[1]].append(d['answers'])
types
names = dd(list)
for d in QA_data:
    if re.match("Jak nazyw", d['question']):
        names[" ".join(d['question'].split(" ")[3:5])].append(d['answers'])
names

# could look for synonims (similar embeddings)
for d in QA_data:
    if re.match(r".* czy ", d['question']):
        if sum([d['answers'][0] in p for p in d['question'].split(" czy ")]) == 0:
            print(d['question'].split("czy"), d['answers'])
for d in QA_data:
    if re.match("Ile ", d['question']):
        for a in d['answers']:
            cc = 0
            try:
                float(a.replace(",", "."))
            except:
                cc += 1
        if cc == len(d['answers']):
            print(d['answers'])
for d in QA_data:
    if re.match("Kto ", d['question']) or re.match(".* kto ", d['question']) or re.match("Kogo ",
                                                                                         d['question']) or re.match(
            ".* kto ", d['question']):
        print(d['answers'])